In [0]:
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np

from collections import Counter
import os
import nltk

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls "/content/drive/My Drive/data"

Artificial_intelligence.yml  ExpectedResults.txt    Sport_games.yml
computers.yml		     IT.yml		    TextQuestions.txt
CourseAllocationCS.yml	     space_and_science.yml  website.yml


In [0]:
alphas = 'abcdefghijklmnopqrstuvwxyz1234567890'

MAX_INPUT_SEQ_LENGTH = 40
MAX_TARGET_SEQ_LENGTH = 40
DATA_DIR_PATH = '/content/gdrive/My Drive/data'
MAX_VOCAB_SIZE = 30000

marker_start = '<begin>'
marker_end = '<end>'
marker_unknown = '<unk>'
marker_pad = '<pad>'

# standard step - reset computation graphs
tf.reset_default_graph()

# 2 more for start and stop markers
input_seq_len = 15
output_seq_len = input_seq_len+2


In [0]:
#defines permissible characters for the chatbot
def permissible_chars(word):

    for char in word:
        if char in alphas:
            return True

    return False

In [0]:

# Compute softmax values for each sets of scores in x.
def softmax(x): 
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()


In [0]:
# To parse the input yml files and create word2index and index2word mappings
import nltk
nltk.download('punkt')
target_counter = Counter()
input_counter = Counter()

input_texts = []
target_texts = []

# Parser base code of GuntherCox dataset obtained from link below and modified as per requirement.
# https://github.com/kushagra2101/ChatCrazie/blob/master/train_seq2seq.py

for file in os.listdir('/content/drive/My Drive/data'):
    filepath = os.path.join('/content/drive/My Drive/data', file)
    if os.path.isfile(filepath):
        print('processing file: ', file)
        lines = open(filepath, 'rt', encoding='utf8').read().split('\n')
        prev_words = []
        for line in lines:

            if line.startswith('- - '):
                prev_words = []

            if line.startswith('- - ') or line.startswith('  - '):
                line = line.replace('- - ', '')
                line = line.replace('  - ', '')
                next_words = [w.lower() for w in nltk.word_tokenize(line)]
                next_words = [w for w in next_words if permissible_chars(w)]
                if len(next_words) > MAX_TARGET_SEQ_LENGTH:
                    next_words = next_words[0:MAX_TARGET_SEQ_LENGTH]

                if len(prev_words) > 0:
                    input_texts.append(prev_words)
                    for w in prev_words:
                        input_counter[w] += 1

                    target_words = next_words[:]
                    for w in target_words:
                        target_counter[w] += 1
                    target_texts.append(target_words)

                prev_words = next_words


for idx, (input_words, target_words) in enumerate(zip(input_texts, target_texts)):
    if idx < 20:
        print([input_words, target_words])

input_w2i, input_i2w, target_w2i, target_i2w = {},{},{},{}

### Creating Word2index and Index2word, forward and reverse mapping ###
## we will create dictionaries to provide a unique integer for each word.
input_w2i[marker_unknown] = 0
input_w2i[marker_pad] = 1
# filter out the rare words
for idx, word in enumerate(input_counter.most_common(MAX_VOCAB_SIZE)):
    input_w2i[word[0]] = idx+2

# inverse dictionary for vocab_to_int.
input_i2w = dict([(idx, word) for word, idx in input_w2i.items()])

## we will create dictionaries to provide a unique integer for each word.
target_w2i[marker_unknown] = 0
target_w2i[marker_pad] = 1
target_w2i[marker_start] = 2
target_w2i[marker_end] = 3
for idx, word in enumerate(target_counter.most_common(MAX_VOCAB_SIZE)):
    target_w2i[word[0]] = idx+4

# inverse dictionary for vocab_to_int.
target_i2w = dict([(idx, word) for word, idx in target_w2i.items()])


###########################################

# inputVocabLen = len(input_word2idx)
# targetVocabLen = len(target_word2idx)

###########################################

# if the word is not found then default with 0. 
# 0 in index means the word is unknown (<unk>)
x = [[input_w2i.get(word, 0) for word in sentence] for sentence in input_texts]
y = [[target_w2i.get(word, 0) for word in sentence] for sentence in target_texts]

inputVocabLen = len(input_w2i)
targetVocabLen = len(target_w2i)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
processing file:  CourseAllocationCS.yml
processing file:  ExpectedResults.txt
processing file:  TextQuestions.txt
processing file:  website.yml
processing file:  Artificial_intelligence.yml
processing file:  IT.yml
processing file:  Sport_games.yml
processing file:  computers.yml
processing file:  space_and_science.yml
[['what', 'courses', 'are', 'offered', 'by', 'mr.', 'hassan', 'mustafa'], ['mr.', 'hassan', 'mustafa', 'is', 'teaching', 'object', 'oriented', 'programming', 'cs217', 'to', 'section', 'a', 'batch', '2018']]
[['mr.', 'hassan', 'mustafa', 'is', 'teaching', 'object', 'oriented', 'programming', 'cs217', 'to', 'section', 'a', 'batch', '2018'], ['mr.', 'hassan', 'mustafa', 'is', 'teaching', 'object', 'oriented', 'programming', 'cs217', 'to', 'section', 'b', 'batch', '2018']]
[['mr.', 'hassan', 'mustafa', 'is', 'teaching', 'object', 'oriented', 'programming', 'cs217', 'to'

In [0]:

# Pad all the sequences to same length
for i in range(len(x)):

    if (len(x[i]) > input_seq_len):
        x[i] = x[i][:input_seq_len-1]

    # Fill in with padding marker
    for k in range(input_seq_len - len(x[i])):
        x[i] = x[i] + [input_w2i[marker_pad]]
            
    if (len(y[i]) > output_seq_len-2):
        y[i] = y[i][:output_seq_len-3]

    # Add end and begin marker
    y[i] = [target_w2i[marker_start]] + y[i] + [target_w2i[marker_end]]

    # Fill in with padding marker
    for k in range(output_seq_len - len(y[i])):
        y[i] = y[i] + [input_w2i[marker_pad]]

    if (i < 10):
        print(x[i])
        print(y[i])

        
# Train Test Split
#X_train,  X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.0)
X_train=x;
Y_train=y;
print(len(X_train))
print(len(Y_train))

[3, 25, 7, 11, 20, 13, 37, 133, 1, 1, 1, 1, 1, 1, 1]
[2, 15, 42, 193, 4, 18, 80, 81, 62, 82, 7, 19, 8, 20, 36, 3, 1]
[13, 37, 133, 2, 28, 93, 94, 56, 95, 8, 29, 10, 30, 51, 1]
[2, 15, 42, 193, 4, 18, 80, 81, 62, 82, 7, 19, 102, 20, 36, 3, 1]
[13, 37, 133, 2, 28, 93, 94, 56, 95, 8, 29, 134, 30, 51, 1]
[2, 15, 42, 193, 4, 18, 80, 81, 62, 82, 7, 19, 103, 20, 36, 3, 1]
[3, 25, 7, 11, 20, 34, 351, 352, 1, 1, 1, 1, 1, 1, 1]
[2, 38, 496, 497, 4, 18, 80, 81, 62, 82, 7, 19, 144, 20, 36, 3, 1]
[3, 25, 7, 11, 20, 14, 85, 96, 174, 1, 1, 1, 1, 1, 1]
[2, 16, 123, 145, 275, 4, 18, 80, 81, 62, 82, 7, 19, 124, 20, 36, 3]
[14, 85, 96, 174, 2, 28, 93, 94, 56, 95, 8, 29, 97, 30, 51]
[2, 16, 123, 145, 275, 4, 18, 80, 81, 62, 82, 7, 19, 276, 20, 36, 3]
[3, 25, 7, 11, 20, 13, 86, 37, 1, 1, 1, 1, 1, 1, 1]
[2, 15, 125, 42, 4, 18, 80, 81, 62, 82, 7, 19, 104, 20, 36, 3, 1]
[3, 25, 7, 11, 20, 13, 86, 37, 1, 1, 1, 1, 1, 1, 1]
[2, 15, 125, 42, 4, 18, 80, 81, 62, 82, 7, 19, 104, 20, 36, 3, 1]
[13, 86, 37, 2, 28, 93,

In [0]:
# Stub code sourced from Neural machine translator for English2German translation
# https://github.com/Nemzy/language-translation. Modified to suit requirements.

# feed data into placeholders
def feed_dict(x, y, batch_size = 64):
    feed = {}
    
    idxes = np.random.choice(len(x), size = batch_size, replace = False)
    
    for i in range(input_seq_len):
        feed[encoder_inputs[i].name] = np.array([x[j][i] for j in idxes], dtype = np.int32)
        
    for i in range(output_seq_len):
        feed[decoder_inputs[i].name] = np.array([y[j][i] for j in idxes], dtype = np.int32)
        
    feed[targets[len(targets)-1].name] = np.full(shape = [batch_size], fill_value = target_w2i[marker_pad], dtype = np.int32)
    
    for i in range(output_seq_len-1):
        batch_weights = np.ones(batch_size, dtype = np.float32)
        target = feed[decoder_inputs[i+1].name]
        for j in range(batch_size):
            if target[j] == target_w2i[marker_pad]:
                batch_weights[j] = 0.0
        feed[target_weights[i].name] = batch_weights
        
    feed[target_weights[output_seq_len-1].name] = np.zeros(batch_size, dtype = np.float32)
    
    return feed

# define our loss function

# sampled softmax loss - returns: A batch_size 1-D tensor of per-example sampled softmax losses
def sampled_loss(labels, logits):
    return tf.nn.sampled_softmax_loss(
                        weights = w_t,
                        biases = b,
                        labels = tf.reshape(labels, [-1, 1]),
                        inputs = logits,
                        num_sampled = 512,
                        num_classes = targetVocabLen)

# decode output sequence
def decode_output(output_seq):
    words = []
    for i in range(output_seq_len):
        smax = softmax(output_seq[i])
        idx = np.argmax(smax)
        words.append(target_i2w[idx])
    return words

In [0]:
# Stub code sourced from Neural machine translator for English2German translation
# https://github.com/Nemzy/language-translation. Modified to suit requirements.

# Defining placeholders
# The first None means the batch size, and the batch size is unknown since user can set it. 
# The second None means the lengths of sentences. 

encoder_inputs = [tf.placeholder(dtype = tf.int32, shape = [None], name = 'encoder{}'.format(i)) 
                  for i in range(input_seq_len)]
decoder_inputs = [tf.placeholder(dtype = tf.int32, shape = [None], name = 'decoder{}'.format(i)) 
                  for i in range(output_seq_len)]

targets = [decoder_inputs[i+1] for i in range(output_seq_len-1)]
# add one more target
targets.append(tf.placeholder(dtype = tf.int32, shape = [None], name = 'last_target'))
target_weights = [tf.placeholder(dtype = tf.float32, shape = [None], 
                                 name = 'target_w{}'.format(i)) for i in range(output_seq_len)]

# output projection
size = 512
w_t = tf.get_variable('proj_w', [targetVocabLen, size], tf.float32)
b = tf.get_variable('proj_b', [targetVocabLen], tf.float32)
w = tf.transpose(w_t)
output_projection = (w, b)

outputs, states = tf.contrib.legacy_seq2seq.embedding_attention_seq2seq(
                                            encoder_inputs,
                                            decoder_inputs,
                                            tf.contrib.rnn.BasicLSTMCell(size),
                                            num_encoder_symbols = inputVocabLen,
                                            num_decoder_symbols = targetVocabLen,
                                            embedding_size = 100,
                                            feed_previous = False,
                                            output_projection = output_projection,
                                            dtype = tf.float32)

# Weighted cross-entropy loss for a sequence of logits
loss = tf.contrib.legacy_seq2seq.sequence_loss(outputs, targets, target_weights, softmax_loss_function = sampled_loss)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
# Stub code sourced from Neural machine translator for English2German translation
# https://github.com/Nemzy/language-translation. Modified to suit requirements.

# ops and hyperparameters
learning_rate = 7e-3
batch_size = 96
steps = 10000
#num_epochs = 2

# ops for projecting outputs
outputs_proj = [tf.matmul(outputs[i], output_projection[0]) + output_projection[1] for i in range(output_seq_len)]

# training op
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)
# tf.train.RMSPropOptimizer

# init op
init = tf.global_variables_initializer()


# Loss values appended to plot diagram
losses = []

# Save checkpoint to restore the model later 
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)

    for step in range(steps):
        feed = feed_dict(X_train, Y_train, batch_size)
        sess.run(optimizer, feed_dict = feed)
        
        if step % 500 == 0:
            loss_value = sess.run(loss, feed_dict = feed)
            print('step: {}, loss: {}'.format(step, loss_value))
            losses.append(loss_value)
            
    saver.save(sess, 'checkpoints/', global_step=step)
    print('Checkpoint is saved')
    
# plot the losses
with plt.style.context('fivethirtyeight'):
    plt.plot(losses, linewidth = 1)
    plt.xlabel('Steps')
    plt.ylabel('Losses')
plt.show()

step: 9500, loss: 4.131120681762695
step: 9500, loss: 4.131120681762695


In [0]:
# To predict response (inference) use the same model as defined above with forward feed

def generateReply(humanMsg):
    import tensorflow as tf
    
    if (len(humanMsg) == 0):
        return ''
    
    with tf.Graph().as_default():
        
        replyMsg = ""

        # same format as in model building
        encoder_inputs = [tf.placeholder(dtype = tf.int32, shape = [None], 
                                         name = 'encoder{}'.format(i)) for i in range(15)]
        decoder_inputs = [tf.placeholder(dtype = tf.int32, shape = [None], 
                                         name = 'decoder{}'.format(i)) for i in range(17)]

        # output projection
        size = 512
        w_t = tf.get_variable('proj_w', [targetVocabLen, size], tf.float32)
        b = tf.get_variable('proj_b', [targetVocabLen], tf.float32)
        w = tf.transpose(w_t)
        output_projection = (w, b)

        # feed_previous is set to true so that output at time t can be fed as input at time t+1
        outputs, states = tf.contrib.legacy_seq2seq.embedding_attention_seq2seq(
                                                    encoder_inputs,
                                                    decoder_inputs,
                                                    tf.contrib.rnn.BasicLSTMCell(size),
                                                    num_encoder_symbols = inputVocabLen,
                                                    num_decoder_symbols = targetVocabLen,
                                                    embedding_size = 100,
                                                    feed_previous = True,
                                                    output_projection = output_projection,
                                                    dtype = tf.float32)
        # ops for projecting outputs
        outputs_proj = [tf.matmul(outputs[i], 
                        output_projection[0]) + output_projection[1] for i in range(output_seq_len)]

        ## Clean and Format incoming msg by humans.
        ## It is better to do the same clean/format as the data preprocessing steps
        ## for the algorithm to predict next words more accurately
        msgLowerCase = [w.lower() for w in nltk.word_tokenize(humanMsg)]
        msg = [w for w in msgLowerCase if permissible_chars(w)]
        if len(msg) > input_seq_len:
            msg = msg[0:input_seq_len-1]

        human_msg_encoded = [input_w2i.get(word, 0) for word in msg]

        # Fill in with padding marker
        for k in range(input_seq_len - len(human_msg_encoded)):
            human_msg_encoded = human_msg_encoded + [input_w2i[marker_pad]]

        # restore all variables - use the last checkpoint saved
        saver = tf.train.Saver()
        path = tf.train.latest_checkpoint('checkpoints')

        with tf.Session() as sess:
            # restore
            saver.restore(sess, path)

            # feed data into placeholders
            feed = {}
            for i in range(input_seq_len):
                feed[encoder_inputs[i].name] = np.array([human_msg_encoded[i]], dtype = np.int32)

            feed[decoder_inputs[0].name] = np.array([target_w2i[marker_start]], dtype = np.int32)

            # translate
            output_sequences = sess.run(outputs_proj, feed_dict = feed)

            ouput_seq = [output_sequences[j][0] for j in range(output_seq_len)]
            #decode output sequence
            words = decode_output(ouput_seq)

            for i in range(len(words)):
                if words[i] not in [marker_end, marker_pad, marker_start]:
                    replyMsg += words[i] + ' '
                             
        #print(replyMsg)
        return replyMsg

In [0]:
bot_reply = generateReply('When is AI class on Monday?')
print(bot_reply)

In [0]:
!ls "/content/drive/My Drive/data"

In [0]:
import numpy as np
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
testquestions_docs=[]
with open ('/content/drive/My Drive/TestQuestions.txt') as f:
    tokens = sent_tokenize(f.read())
    for line in tokens:
        testquestions_docs.append(line)

print("Number of test questions:",len(testquestions_docs))

In [0]:
expectedanswers_docs=[]
with open ('/content/drive/My Drive/TestAnswers.txt') as f:
    tokens = sent_tokenize(f.read())
    for line in tokens:
        expectedanswers_docs.append(line)

print("Number of test questions:",len(expectedanswers_docs))

**CALCULATING BLEU SCORE**

In [0]:
sum=0
for i in range(0,9,1):
  Question = testquestions_docs[i]
  print("Question:")
  print(Question)
  expected_data = expectedanswers_docs[i].lower()
  print("Expected response:")
  print( expected_data)
  actual_data = generateReply(Question)
  print("Actual response:")
  print( actual_data)
  actual_response = word_tokenize(actual_data)
  expected_response = word_tokenize(expected_data)
  BLEUscore = nltk.translate.bleu_score.sentence_bleu([expected_response], actual_response, weights = (0.5, 0.5))
  print("BLEU SCORE FOR RESPONSE " +str(i)+": "+str(BLEUscore))
  sum=sum+BLEUscore

In [0]:
#option = ""

#while(option !="yes"):
#name1 = input("Enter Question : ") 
  #bot_reply = generateReply(name1)
  #print("To end conversation enter yes else enter no")
  #option = input("Enter option : ")

In [0]:
sum=sum/10
print("BLEU SCORE: "+str(sum))

In [0]:
import numpy as np
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
actual_response = word_tokenize("I am Maham")
expected_response = word_tokenize("I am is Maham")
BLEUscore = nltk.translate.bleu_score.sentence_bleu([expected_response], actual_response, weights = (0.5, 0.5))
print(BLEUscore)